In order to run the code that does the basic steps run the cells numbered with hashtags

First the imports

In [29]:
#1
import os 

import pandas as pd
import numpy as np
import csv
import calendar

from statistics import mean, mode

# Exploration & Cleaning:

We want to have the data in an accessible form. This is done by reading the data into a dictionary of panada dataframes. Each element in the dictionary has a key (which is the name of the month the data was collected in) and a value (the csv entries in a panda dataframe) 

In [2]:
#2
# returns a dictionary containing the data from the .csv files
def read_data(dir):
    dfs = {}
    files = os.listdir(dir)

    # filter files by .csv bec some are google sheets
    # extract the number of the month, then sort by month so that they are then stored in order of month
    csv_files = sorted([int(file[4:6]) for file in files if file.endswith('.csv')])
    
    for csv_file in csv_files:
        # reconstruct the name of file bec we extracted the month number to sort them by month
        reconstructed_name = '2023'+str(csv_file)+'-divvy-tripdata.csv' if csv_file >= 10 else '20230'+str(csv_file)+'-divvy-tripdata.csv'
        f = os.path.join(dir, reconstructed_name) 
        
        # get month name
        month = calendar.month_name[csv_file]
        
        # read csv file into dataframe
        dfs[month] = pd.read_csv(f)

    return dfs

In [3]:
#3
original_dir = "../../BikeShareData/OriginalData"
data = read_data(original_dir)

In [4]:
# no of files should be 11
no_of_files = len(data)
print(no_of_files)

11


In [51]:
month_names = data.keys()
print(month_names)

dict_keys(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November'])


In [52]:
# the first and last 5 entries
#data["January"]
#data["February"]
#data["March"]
#data["April"]
data["May"]
#data["June"]
#data["July"]
#data["August"]
#data["September"]
#data["October"]
#data["November"]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0D9FA920C3062031,electric_bike,2023-05-07 19:53:48,2023-05-07 19:58:32,Southport Ave & Belmont Ave,13229,NaN,NaN,41.939408,-87.663831,41.930000,-87.650000,member
1,92485E5FB5888ACD,electric_bike,2023-05-06 18:54:08,2023-05-06 19:03:35,Southport Ave & Belmont Ave,13229,NaN,NaN,41.939482,-87.663848,41.940000,-87.690000,member
2,FB144B3FC8300187,electric_bike,2023-05-21 00:40:21,2023-05-21 00:44:36,Halsted St & 21st St,13162,NaN,NaN,41.853793,-87.646719,41.860000,-87.650000,member
3,DDEB93BC2CE9AA77,classic_bike,2023-05-10 16:47:01,2023-05-10 16:59:52,Carpenter St & Huron St,13196,Damen Ave & Cortland St,13133,41.894556,-87.653449,41.915983,-87.677335,member
4,C07B70172FC92F59,classic_bike,2023-05-09 18:30:34,2023-05-09 18:39:28,Southport Ave & Clark St,TA1308000047,Southport Ave & Belmont Ave,13229,41.957081,-87.664199,41.939478,-87.663748,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604822,48BDA26F34445546,electric_bike,2023-05-18 10:26:43,2023-05-18 10:48:00,Clark St & Elmdale Ave,KA1504000148,NaN,NaN,41.990876,-87.669721,42.000000,-87.660000,member
604823,573025E5EDE10DE1,electric_bike,2023-05-17 14:32:48,2023-05-17 14:45:37,State St & 33rd St,13216,NaN,NaN,41.834734,-87.625798,41.830000,-87.620000,member
604824,D88D48898C6FB63E,electric_bike,2023-05-17 07:59:29,2023-05-17 08:04:54,Columbus Dr & Randolph St,13263,NaN,NaN,41.884422,-87.619393,41.880000,-87.630000,member
604825,4692DCD2F87497F5,electric_bike,2023-05-18 08:34:48,2023-05-18 08:38:40,Public Rack - Karlov Ave & Lawrence Ave,1127.0,NaN,NaN,41.970000,-87.730000,41.970000,-87.740000,member


In [57]:
data["May"].dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [54]:
# the number of unique values in May
# unique values for specific columns
print("Column Name:          NUnique")
print(data['May'].nunique())
print(data['May'].rideable_type.unique())
print(data['May'].member_casual.unique())

Column Name:          NUnique
ride_id               604827
rideable_type              3
started_at            503683
ended_at              505259
start_station_name      1287
start_station_id        1250
end_station_name        1254
end_station_id          1210
start_lat             188591
start_lng             185410
end_lat                 4759
end_lng                 4762
member_casual              2
dtype: int64
['electric_bike' 'classic_bike' 'docked_bike']
['member' 'casual']


In [35]:
# total number of entries
total = data["May"].ride_id.nunique()
# number of unique started at
started = data["May"].started_at.nunique()
print("Total: \t",total)
print("Started_At: ",started)
print("Percent: ",(total-started)*100/total)

# how many times was the started_at repeated
duplicated_col = data["May"].started_at.duplicated()
print("Started_At: ",duplicated_col.sum()+data["May"].started_at.nunique())
get_values = data["May"][duplicated_col]
# get first value in the duplicates
first_entry = data["May"][duplicated_col].iloc[0].started_at


Total: 	 604827
Started_At:  503683
Percent:  16.72279842004408
Started_At:  604827


In [33]:
# find the entry that has the same start_at as the first one retrieved from the duplicates
data["May"][data["May"].started_at == first_entry]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
727,041763A703C94783,electric_bike,2023-05-28 14:59:58,2023-05-28 15:12:47,Kedzie Ave & Milwaukee Ave,13085,Kilpatrick Ave & Parker Ave,358,41.929673,-87.708045,41.930731,-87.744106,casual
2710,5AEC034DB275854E,electric_bike,2023-05-28 14:59:58,2023-05-28 15:26:56,Broadway & Belmont Ave,13277,NaN,NaN,41.940170,-87.645626,41.960000,-87.640000,casual
400665,A99D22D37DC92962,electric_bike,2023-05-28 14:59:58,2023-05-28 15:09:35,NaN,NaN,MTV Hubbard St,021320,41.880000,-87.660000,41.889779,-87.680341,member
557920,79A55702C0B6D246,docked_bike,2023-05-28 14:59:58,2023-05-28 16:24:12,Streeter Dr & Grand Ave,13022,Field Museum,13029,41.892278,-87.612043,41.865312,-87.617867,casual


The count_entries method will do the following:
 * calculate number of entires in each file
 * calculate number of columns in each file
 * calculate the average and total number of entries across all files
 * write these outputs to a file
 * Option to remove duplicates

In [4]:
#4
def count_entries(dataset, file_name, flag):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Month","No Of Entries","No Of Cols"])

        total = 0

        for month, df in dataset.items():
            if flag:
                df = df.drop_duplicates()
                
            entries = len(df)
            writer.writerow([month, entries,len(df.columns)])
            total += len(df)
            print(f"For {month}:{df.shape}")

        average = int(total/11)
        writer.writerow(["Total:", total])
        writer.writerow(["Average:", average])
        print(f"Average number of entries per file: {average}")
        print(f"Total number of entries across all files: {total}")


In [5]:
#5
count_entries(data, "BikeRides_Original.csv", False)

For January:(190301, 13)
For February:(190445, 13)
For March:(258678, 13)
For April:(426590, 13)
For May:(604827, 13)
For June:(719618, 13)
For July:(767650, 13)
For August:(771693, 13)
For September:(666371, 13)
For October:(537113, 13)
For November:(362518, 13)
Average number of entries per file: 499618
Total number of entries across all files: 5495804


In [77]:
count_entries(data, "BikeRides_without_Duplicates.csv", True)

For January:(190301, 13)
For February:(190445, 13)
For March:(258678, 13)
For April:(426590, 13)
For May:(604827, 13)
For June:(719618, 13)
For July:(767650, 13)
For August:(771693, 13)
For September:(666371, 13)
For October:(537113, 13)
For November:(362518, 13)
Average number of entries per file: 499618
Total number of entries across all files: 5495804


From the above print out, it is clear that all the files have the same number of columns. So that is a good preliminary check on the consistency of the data across the files. And we can see that in total we have 5.5 Million entries, with an average of 500000 entries per month. All calculated numbers before and after removing duplicates are identical, so the original dataset did not have any duplicates.

Next we will look at the percentage of nulls across the columns:

In [6]:
#6
def check_NAN(dataset, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        for month, df in dataset.items():
            if month == 'January':
                # if the month is January add the word "Month" to column names
                column_names = df.columns.insert(0, "Month")
                writer.writerow(column_names)
            
            percentage = df.isna().sum()*100/len(df)
            y = percentage.apply(lambda x: str(int(x))+"%" if x > 1 else ("< 1%" if x > 0 else str(0))).values
            values = np.insert(y, 0, month)
            print(values)
            writer.writerow(values)
        

In [7]:
#7
check_NAN(data, "NaN_Percentages_Orig.csv")

['January' '0' '0' '0' '0' '14%' '14%' '14%' '14%' '0' '0' '< 1%' '< 1%'
 '0']
['February' '0' '0' '0' '0' '13%' '13%' '14%' '14%' '0' '0' '< 1%' '< 1%'
 '0']
['March' '0' '0' '0' '0' '13%' '13%' '14%' '14%' '0' '0' '< 1%' '< 1%' '0']
['April' '0' '0' '0' '0' '14%' '14%' '16%' '16%' '0' '0' '< 1%' '< 1%' '0']
['May' '0' '0' '0' '0' '14%' '14%' '15%' '15%' '0' '0' '< 1%' '< 1%' '0']
['June' '0' '0' '0' '0' '16%' '16%' '17%' '17%' '0' '0' '< 1%' '< 1%' '0']
['July' '0' '0' '0' '0' '16%' '16%' '16%' '16%' '0' '0' '< 1%' '< 1%' '0']
['August' '0' '0' '0' '0' '15%' '15%' '16%' '16%' '0' '0' '< 1%' '< 1%'
 '0']
['September' '0' '0' '0' '0' '15%' '15%' '16%' '16%' '0' '0' '< 1%' '< 1%'
 '0']
['October' '0' '0' '0' '0' '15%' '15%' '16%' '16%' '0' '0' '< 1%' '< 1%'
 '0']
['November' '0' '0' '0' '0' '15%' '15%' '15%' '15%' '0' '0' '< 1%' '< 1%'
 '0']


Drop the columns with NAN values, and convert the started and ended at to datetime.

In [8]:
#8
# 1st phase of clean data with the ended_before_started column to be able to see and count the entries where this is True
def clean_data1(dataset):
    col_to_drop = ['ride_id', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
    clean_data = {}
    ended_before_started = {}

    for month, df in dataset.items():
        #print("shape before",df.shape)
        clean_data[month] = df.drop(columns = col_to_drop) # drop columns
        #print("shape after",df.shape)
        #df.fillna('') # fill NaN
        #df.dropnna(subset = "Column Name", inplace=True) # drop rows where column_name has NaN

        # convert columns to datetime 
        clean_data[month].started_at = pd.to_datetime(df['started_at'])
        clean_data[month].ended_at   = pd.to_datetime(df['ended_at'])

        ended_before_started[month] = clean_data[month].started_at > clean_data[month].ended_at
        
    return clean_data, ended_before_started

In [21]:
#9
cleaned_data1, ended_before_started = clean_data1(data)

In [82]:
cleaned_data1['January'].info()
cleaned_data1['January'].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190301 entries, 0 to 190300
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   rideable_type  190301 non-null  object        
 1   started_at     190301 non-null  datetime64[ns]
 2   ended_at       190301 non-null  datetime64[ns]
 3   member_casual  190301 non-null  object        
dtypes: datetime64[ns](2), object(2)
memory usage: 5.8+ MB


,rideable_type,started_at,ended_at,member_casual
0,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,member
1,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,member
2,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,casual
3,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,member
4,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,member


In [83]:
month = 'May'
cleaned_data1[month][ended_before_started[month]]

,rideable_type,started_at,ended_at,member_casual
8308,classic_bike,2023-05-29 17:34:21,2023-05-29 17:34:09,member
38552,electric_bike,2023-05-29 16:57:34,2023-05-29 16:57:27,casual
103546,electric_bike,2023-05-26 15:39:47,2023-05-26 15:38:17,member
103547,electric_bike,2023-05-26 15:38:53,2023-05-26 15:38:17,member
209340,classic_bike,2023-05-07 15:54:58,2023-05-07 15:54:47,casual
211708,classic_bike,2023-05-23 17:39:38,2023-05-23 17:39:35,casual
216859,classic_bike,2023-05-13 18:08:15,2023-05-13 18:08:09,member
336480,electric_bike,2023-05-29 11:31:41,2023-05-29 11:31:33,member
417351,classic_bike,2023-05-27 05:31:51,2023-05-27 05:31:37,member
456170,electric_bike,2023-05-30 07:40:55,2023-05-30 07:39:58,member


In [84]:
# calculates total number of entries where ended_before_started == True
total = []
for month, df in cleaned_data1.items():
    total.append(ended_before_started[month].sum())

print(total, sum(total)) 

[0, 1, 0, 4, 10, 7, 30, 60, 50, 36, 64] 262


In [22]:
#10
#2nd version of clean data that will drop the rows where ended > started
def clean_data2(dataset, flag):
    clean_data = {}

    for month, df in dataset.items():
        clean_data[month] = df[flag[month] == False]
        
    return clean_data

In [23]:
#11
cleaned_data = clean_data2(cleaned_data1, ended_before_started)

In [24]:
#12
# check no of columns and entries after removing
count_entries(cleaned_data, "BikeRides_Cleaned.csv", False)

For January:(190301, 4)
For February:(190444, 4)
For March:(258678, 4)
For April:(426586, 4)
For May:(604817, 4)
For June:(719611, 4)
For July:(767620, 4)
For August:(771633, 4)
For September:(666321, 4)
For October:(537077, 4)
For November:(362454, 4)
Average number of entries per file: 499594
Total number of entries across all files: 5495542


In [25]:
#13
check_NAN(cleaned_data, "NaN_Percentages_Clean.csv")

['January' '0' '0' '0' '0']
['February' '0' '0' '0' '0']
['March' '0' '0' '0' '0']
['April' '0' '0' '0' '0']
['May' '0' '0' '0' '0']
['June' '0' '0' '0' '0']
['July' '0' '0' '0' '0']
['August' '0' '0' '0' '0']
['September' '0' '0' '0' '0']
['October' '0' '0' '0' '0']
['November' '0' '0' '0' '0']


# Preparation:

In [40]:
#14
# converts seconds from deltatime into HH:MM:SS format or DD days HH:MM:SS if days = True
def format_time(seconds, days= False):
    min, sec    = divmod(seconds, 60)
    hour, min   = divmod(min, 60)

    if days:
        day, hour   = divmod(hour, 24)
        if day == 1:
            return '%d day %d:%02d:%02d' % (day, hour, min, sec)
        elif day > 1:
            return '%d days %d:%02d:%02d' % (day, hour, min, sec)
    
    return '%d:%02d:%02d' % (hour, min, sec)

In [26]:
#15
# add the columns ride_length and day_of_week
def prepare_data(dataset):

    for _, df in dataset.items():
        if 'ride_length' not in df:
            df.insert(len(df.columns), "ride_length", df.ended_at - df.started_at)
        else: 
            print("Column ride_length already exists")
        
        if 'day_of_week' not in df:
            df.insert(len(df.columns), "day_of_week", df.started_at.dt.day_name())
        else: 
            print("Column day_of_week already exists")
        
    return dataset

In [27]:
#16
prepared_data = prepare_data(cleaned_data)

In [28]:
prepared_data['May'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 604817 entries, 0 to 604826
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype          
---  ------         --------------   -----          
 0   rideable_type  604817 non-null  object         
 1   started_at     604817 non-null  datetime64[ns] 
 2   ended_at       604817 non-null  datetime64[ns] 
 3   member_casual  604817 non-null  object         
 4   ride_length    604817 non-null  timedelta64[ns]
 5   day_of_week    604817 non-null  object         
dtypes: datetime64[ns](2), object(3), timedelta64[ns](1)
memory usage: 32.3+ MB


In [101]:
prepared_data['May']

,rideable_type,started_at,ended_at,member_casual,ride_length,day_of_week
0,electric_bike,2023-05-07 19:53:48,2023-05-07 19:58:32,member,0 days 00:04:44,Sunday
1,electric_bike,2023-05-06 18:54:08,2023-05-06 19:03:35,member,0 days 00:09:27,Saturday
2,electric_bike,2023-05-21 00:40:21,2023-05-21 00:44:36,member,0 days 00:04:15,Sunday
3,classic_bike,2023-05-10 16:47:01,2023-05-10 16:59:52,member,0 days 00:12:51,Wednesday
4,classic_bike,2023-05-09 18:30:34,2023-05-09 18:39:28,member,0 days 00:08:54,Tuesday
...,...,...,...,...,...,...
604822,electric_bike,2023-05-18 10:26:43,2023-05-18 10:48:00,member,0 days 00:21:17,Thursday
604823,electric_bike,2023-05-17 14:32:48,2023-05-17 14:45:37,member,0 days 00:12:49,Wednesday
604824,electric_bike,2023-05-17 07:59:29,2023-05-17 08:04:54,member,0 days 00:05:25,Wednesday
604825,electric_bike,2023-05-18 08:34:48,2023-05-18 08:38:40,member,0 days 00:03:52,Thursday


In [36]:
# testing of the method format_time
or_val = prepared_data["May"].ride_length.max()
print("Orig val: ",or_val)
print("Orig val seconds: ",or_val.seconds)
print("Orig val seconds and days: ",or_val.seconds + or_val.days*24*60*60)
print("Formated val with seconds only: ",format_time(or_val.seconds))
print("Formated val with seconds, days together: ",format_time(or_val.seconds+or_val.days*24*60*60))
print("Formated val with seconds, days seperate: ",format_time(or_val.seconds+or_val.days*24*60*60, True))

Orig val:  20 days 06:50:31
Orig val seconds:  24631
Orig val seconds and days:  1752631
Formated val with seconds only:  6:50:31
Formated val with seconds, days together:  486:50:31
Formated val with seconds, days seperate:  20 days 6:50:31


In [112]:
#17
# if split: then calculate statistics based on member_casual
def statistics(dataset, file_name, split = False, split_into = ['all'], split_by = 'member_casual'):

    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)

        statistics = {}
        operations = ['mean', 'max', 'mode']
        df_list = {}

        if split:
            #  make a list with the values that we want to split into, and repeat it 3 times, bec mean, max and mode
            line = [splitter for splitter in split_into*3]
            # add empty entry in the begining for motnh
            line.insert(0, "")
            writer.writerow(line)
        else:
            writer.writerow(["Month","Mean Ride Length","Max Ride Length", "Mode of Day of Week"])
        
        for type in split_into:
            #statistics[type] = {'meanlist' : [], 'maxlist' : [], 'modelist' : []}
            statistics[type] = {op : [] for op in operations}
        
        print(statistics)
        return
        i = 0
        for month, df in dataset.items():

            for type in split_into:
                # if split then filter df by rider type
                if split:
                    df_list[type] = df[df.split_by == type]
                else:
                    df_list[type] = df
                
                # get mean, max, mode 
                mean_i, max_i, mode_i = df_list[type].ride_length.mean(), df_list[type].ride_length.max(), df_list[type].day_of_week.mode()[0] 
                # convert mean and max into seconds
                mean_i, max_i = mean_i.seconds+(mean_i.days*24*60*60), max_i.seconds+(max_i.days*24*60*60)

                # add each statistic to a list so that in the end we can find statistics across entire dataset
                statistics[type]['meanlist'].append(mean_i)
                statistics[type]['maxlist'].append(max_i)
                statistics[type]['modelist'].append(mode_i)

            # write to file
            if split:
                line = 
                writer.writerow([month, format_time(statistics['member']['meanlist'][i]), format_time(statistics['casual']['meanlist'][i]), 
                                        format_time(statistics['member']['maxlist'][i], True), format_time(statistics['casual']['maxlist'][i], True), 
                                        statistics['member']['modelist'][i], statistics['casual']['modelist'][i]
                                ])
                print([month, format_time(statistics['member']['meanlist'][i]), format_time(statistics['casual']['meanlist'][i]), 
                                        format_time(statistics['member']['maxlist'][i], True), format_time(statistics['casual']['maxlist'][i], True), 
                                        statistics['member']['modelist'][i], statistics['casual']['modelist'][i]
                                ])
            else:
                writer.writerow([month, format_time(mean_i), format_time(max_i, True), mode_i])
                print([month, format_time(mean_i), format_time(max_i, True), mode_i])    

            i += 1
        
        # we want to add the total statistics across whole year
        writer.writerow([""])

        if split:
                writer.writerow(["Across whole year", 
                             format_time(mean(statistics['member']['meanlist'])), format_time(mean(statistics['casual']['meanlist'])) ,
                             format_time(max(statistics['member']['maxlist']), True), format_time(max(statistics['casual']['maxlist']), True), 
                             mode(statistics['member']['modelist']), mode(statistics['casual']['modelist'])])
                print(["Across whole year",format_time(mean(statistics['member']['meanlist'])), format_time(mean(statistics['casual']['meanlist'])) ,format_time(max(statistics['member']['maxlist']), True), format_time(max(statistics['casual']['maxlist']), True), mode(statistics['member']['modelist']), mode(statistics['casual']['modelist'])])
        else:
            writer.writerow(["Across whole year", 
                             format_time(mean(statistics['all']['meanlist'])), 
                             format_time(max(statistics['all']['maxlist']), True), 
                             mode(statistics['all']['modelist'])])
            print(["Across whole year", format_time(mean(statistics['all']['meanlist'])), format_time(max(statistics['all']['maxlist']), True),mode(statistics['all']['modelist'])])
        
        return
        

SyntaxError: invalid syntax (<ipython-input-112-d0390ebd9a27>, line 49)

In [108]:
#19
# for dataset split into memeber and casual
statistics(prepared_data, "Mean_Max_Mode_split.csv", True, split_into = ['member', 'casual'], split_by = 'member_casual')

In [ ]:
#18
# for entire dataset without splitting
statistics(prepared_data, "Mean_Max_Mode.csv")